In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
pip install transformers datasets==3.6.0 evaluate jiwer torchaudio librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 158.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 266.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 127.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 219.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 174.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## CV_17

In [2]:
# import os
# import pandas as pd

# from datasets import Dataset
# df = pd.read_csv("test.tsv", sep='\t')
# df = df[['path','sentence']]

# # Define the folder containing your audio files
# audio_folder = "test_audio"

# # List all files in the folder
# audio_files = set(os.listdir(audio_folder))

# # Filter the DataFrame to include only rows where the 'path' exists in the folder
# df_filtered = df[df['path'].isin(audio_files)]

# # Optionally, save the filtered DataFrame
# df_filtered.to_csv("filtered_dataframe.csv", index=False)
# df_filtered

In [3]:
# dataset = Dataset.from_pandas(df_filtered)
# dataset

In [4]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_17 = load_dataset("mozilla-foundation/common_voice_17_0", "fr", split="test")

In [5]:
dataset = cv_17.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 16159
})

In [5]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Sampling rate: 48000
array:[ 0.00000000e+00 -4.95903318e-12 -4.28973263e-13 ... -5.87302275e-05
 -8.39358181e-05 -7.38822855e-05]


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [6]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [13]:
filtered_data = dataset.select(range(1))
def print_first_audio(batch): 
    first_audio = batch["audio"][0] 
    print(f"Sampling rate: {first_audio['sampling_rate']}")
    print(f"array:{first_audio['array']}")
    return batch
filtered_data.map(print_first_audio, batched=True, batch_size=1)


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 1
})

In [32]:
# filtered_data = dataset.select(range(5))

# from datasets import Audio

# filtered_data = filtered_data.cast_column("audio", Audio(sampling_rate=16000))

In [33]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

# Now transcribed_dataset contains: 'audio', 'sentence', 'transcription'


Device set to use cuda:0


In [34]:
transcriptions

Dataset({
    features: ['audio', 'sentence', 'transcription'],
    num_rows: 16159
})

In [35]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

Ground Truth: Ce dernier a évolué tout au long de l'histoire romaine.
Transcription:  Ce dernier est volé tout au long de l'histoire romaine.
--------------------------------------------------
Ground Truth: Son actionnaire majoritaire est le Conseil territorial de Saint-Pierre-et-Miquelon.
Transcription:  son actionnaire majoritaire et le conseil territorial de Saint-Pierre et Miqueulon
--------------------------------------------------
Ground Truth: Ce site contient quatre tombeaux de la dynastie achéménide et sept des Sassanides.
Transcription:  Ce site contient 4 tombeaux de la dynastie Hemenid et 7 des Sassanid.
--------------------------------------------------
Ground Truth: J'ai dit que les acteurs de bois avaient, selon moi, beaucoup d'avantages sur les autres.
Transcription:  J'ai dit que les acteurs de bois avaient, selon moi, beaucoup davantage sur les autres.
--------------------------------------------------
Ground Truth: Les Pays-Bas ont remporté toutes les éditions.
Trans

## CV_16

In [26]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_16 = load_dataset("mozilla-foundation/common_voice_16_0", "fr", split="test")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

DatasetNotFoundError: Dataset 'mozilla-foundation/common_voice_16_0' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/mozilla-foundation/common_voice_16_0 to ask for access.

In [ ]:
dataset = cv_16.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

## CV_15

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

cv_15 = load_dataset("mozilla-foundation/common_voice_15_0", "fr", split="test")

In [ ]:
dataset = cv_15.remove_columns(['path','client_id','up_votes','down_votes','age','gender','accent','locale','segment','variant'])
dataset

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

## MLS

In [ ]:
from datasets import load_dataset

# Try loading the dataset without streaming
dataset = load_dataset("facebook/multilingual_librispeech", "french",split="test")

dataset

In [ ]:
dataset = dataset["test"].remove_columns(['original_path', 'begin_time', 'end_time', 'audio_duration', 'chapter_id', 'file', 'id'])
dataset

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)

## Fleurs

In [ ]:
from datasets import load_dataset
from huggingface_hub import notebook_login
notebook_login()

fleurs = load_dataset("google/fleurs", "fr_fr", split="test")

In [ ]:
dataset = fleurs.remove_columns

In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
from transformers import pipeline

# Load the ASR pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Batched function to transcribe a batch of examples
def transcribe_batch(batch):
    # Extract audio arrays and sampling rates
    audio_arrays = batch["audio"]
    inputs = [a["array"] for a in audio_arrays]
    sentences = batch["sentence"]
    # Transcribe using the pipeline
    results = pipe(inputs)

    # Extract just the text
    transcriptions = [r["text"] for r in results]

    # Return the transcriptions as a new column
    return {
        "transcription": transcriptions
    }

# Apply the batched function to the dataset
transcriptions = dataset.map(transcribe_batch, batched=True, batch_size=8)  # You can tune batch_size

In [ ]:
from jiwer import wer, cer

total_wer = 0
total_cer = 0

# Store the WER and CER values for averaging later
wer_values = []
cer_values = []

# Now calculate WER and CER for each transcription and store them
for transcription in transcriptions:
    ground_truth = transcription['sentence']
    predicted_text = transcription['transcription']
    
    # Calculate WER and CER for each transcription
    word_error_rate = wer(ground_truth, predicted_text)  # Calculate WER
    char_error_rate = cer(ground_truth, predicted_text)  # Calculate CER
    
    # Append the values to lists for later averaging
    wer_values.append(word_error_rate)
    cer_values.append(char_error_rate)
    
    # Print the top 5 transcriptions based on WER (or you can choose CER as well)
    if len(wer_values) <= 5:
        print(f"Audio Path: {transcription['audio_path']}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Transcription: {predicted_text}")
        print("-" * 50)

# Calculate the average WER and CER
avg_wer = sum(wer_values) / len(wer_values) if wer_values else 0
avg_cer = sum(cer_values) / len(cer_values) if cer_values else 0

# Print the average WER and CER
print("\nAverage WER:", avg_wer)
print("Average CER:", avg_cer)